<a href="https://colab.research.google.com/github/KarrasYiannis/Trading/blob/main/23_7_23.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install binance

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
#from binance.client import Client
from binance import BinanceSocketManager

ImportError: ignored

In [ ]:
# Tutorial's URL:
#https://www.youtube.com/watch?v=w0wKN5L1HYI

In [ ]:
import json
import requests
import pandas as pd
import datetime

In [ ]:
currency_pair = "btcusd"
url = f"https://www.bitstamp.net/api/v2/ohlc/{currency_pair}/"

start = "2021-01-01"
end = "2021-01-02"

dates = pd.date_range(start, end, freq = "1H")
dates = [ int(x.value/10**9) for x in list(dates)]

master_data = []

for first, last in zip(dates, dates[1:]):
    print(first, last)

    params =  {"step": 60,
               "limit": 10,
               "start": first,
               "end": last
               }
    data = requests.get(url, params =params)

    data  = data.json()["data"]["ohlc"]

    master_data += data

df = pd.DataFrame(master_data)
df = df.drop_duplicates()

df["timestamp"] = df["timestamp"].astype(int)
df = df.sort_values(by="timestamp")

#df = df[df["timestamp"] >= dates[0]]
#df = df[df["timestamp"] >= dates[-1]]

df.to_csv("btcusd.csv")

1609459200 1609462800
1609462800 1609466400
1609466400 1609470000
1609470000 1609473600
1609473600 1609477200
1609477200 1609480800
1609480800 1609484400
1609484400 1609488000
1609488000 1609491600
1609491600 1609495200
1609495200 1609498800
1609498800 1609502400
1609502400 1609506000
1609506000 1609509600
1609509600 1609513200
1609513200 1609516800
1609516800 1609520400
1609520400 1609524000
1609524000 1609527600
1609527600 1609531200
1609531200 1609534800
1609534800 1609538400
1609538400 1609542000
1609542000 1609545600


In [ ]:
#Backtesting
!pip  install vectorbt


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.7/526.7 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.8/293.8 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.3 MB/s eta 0:00:00


In [ ]:
from datetime import datetime
import vectorbt as vbt
import numpy as np

In [ ]:
btc_price  = pd.read_csv("btcusd.csv")[['timestamp', "close"]]
btc_price['date'] = pd.to_datetime(btc_price["timestamp"], unit = "s")

btc_price = btc_price.set_index("date")["close"]

rsi = vbt.RSI.run(btc_price, window = 14, short_name= 'rsi')

num=10
entry_points = np.linspace(1,45, num=num)
exit_points = np.linspace(55,99, num=num)

grid  =np.array(np.meshgrid(entry_points, exit_points)).T.reshape(-1,2)

entries =  rsi.rsi_crossed_below(list(grid[:, [0]]))
exits =  rsi.rsi_crossed_above(list(grid[:, [1]]))

pf = vbt.Portfolio.from_signals(btc_price, entries, exits)

metric  = "total_return"

pf_perf = pf.deep_getattr(metric)

pf_perf_matrix = pf_perf.vbt.unstack_to_df(index_levels = "rsi_crossed_above",
                                           column_levels = "rsi_crossed_below")
print(pf_perf)
pf_perf.vbt.heatmap(
    xaxis_title ="exit",
    yaxis_title = "entry"
).show()

rsi_crossed_below  rsi_crossed_above
1.0                55.000000            0.000000
                   59.888889            0.000000
                   64.777778            0.000000
                   69.666667            0.000000
                   74.555556            0.000000
                                          ...   
45.0               79.444444            0.009755
                   84.333333            0.009306
                   89.222222            0.004601
                   94.111111            0.004601
                   99.000000            0.004601
Name: total_return, Length: 100, dtype: float64


In [ ]:
 pf.statics()


AttributeError: ignored